# Create Elastic Search Pipeline

Create and wire containers:
* elasticsearch
* kibana
* filebeats
* logstash

## Create Elasticsearch Container

This section replicates the function of this docker cli command
```bash
docker run -d -p 9200:9200 -p 9300:9300 \
--network=elastic \
--name=elasticsearch \
-v /Users/lou/Documents/elasticsearch/esdata1:/usr/share/elasticsearch/data \
-e "node.name=alacrity.local" \
--restart=unless-stopped docker.elastic.co/elasticsearch/elasticsearch:6.3.2
```

In [4]:
import docker
import socket
import uuid
client = docker.from_env()

In [5]:
# replace these with the appropriate values for your setup
# all components in pipeline should match versions
es_version = '6.3.2'
# configurations and data for docker containers are rooted here
docker_root = '/Users/lou/Documents/docker-python-api-elk-provisioning'

In [6]:
# we need to create a random elastic search container name since we
# can't currently directly create an alias. See https://github.com/docker/docker-py/issues/1571
es_container_name="elasticsearch-{}".format(str(uuid.uuid4()))
es_container_name

'elasticsearch-43d53497-e397-44fe-8e3f-6d00ded827f3'

In [7]:
es_image = "docker.elastic.co/elasticsearch/elasticsearch:{}".format(es_version)
es_container = client.containers.run(
    es_image,
    name=es_container_name,
    detach=True,
    network="elastic",
    ports={
        '9200/tcp': '9200',
        '9300/tcp': '9300'
    },
    volumes={
        "{}/elasticsearch/esdata1".format(docker_root): {
            'bind': '/usr/share/elasticsearch/data',
            'mode': 'rw'
        }
    },
    environment={
        'node.name': socket.gethostname()
    },
    restart_policy={
        'Name': 'unless-stopped'
    }
)


## Create Kibana Container
This section replicates the function of this docker cli command
```bash
docker run -d -p 5601:5601 \
--network=elastic \
-e "ELASTICSEARCH_URL=http://elasticsearch:9200" \
-v /Users/lou/Documents/kibana/kibana.yml:/usr/share/kibana/config/kibana.yml \
--restart=unless-stopped docker.elastic.co/kibana/kibana:6.3.2
```

In [12]:
kb_image = "docker.elastic.co/kibana/kibana:{}".format(es_version)
kb_container = client.containers.run(
    kb_image,
    detach=True,
    network="elastic",
    environment={
        'ELASTICSEARCH_URL': 'http://{}:9200'.format(es_container_name)
    },
    restart_policy={
        'Name': 'unless-stopped'
    },
    ports={
        '5601/tcp': '5601'
    },
    volumes={
        "{}/kibana/kibana.yml".format(docker_root): {
            'bind': '/usr/share/kibana/config/kibana.yml',
            'mode': 'ro'
        }
    }
)

## Shutdown and Clean Up

In [11]:
kb_container.stop()
kb_container.remove()
#es_container.stop()
#es_container.remove()


In [14]:
kb_container.logs()

b'{"type":"log","@timestamp":"2018-08-20T16:40:16Z","tags":["status","plugin:kibana@6.3.2","info"],"pid":1,"state":"green","message":"Status changed from uninitialized to green - Ready","prevState":"uninitialized","prevMsg":"uninitialized"}\n{"type":"log","@timestamp":"2018-08-20T16:40:16Z","tags":["status","plugin:elasticsearch@6.3.2","info"],"pid":1,"state":"yellow","message":"Status changed from uninitialized to yellow - Waiting for Elasticsearch","prevState":"uninitialized","prevMsg":"uninitialized"}\n{"type":"log","@timestamp":"2018-08-20T16:40:16Z","tags":["status","plugin:xpack_main@6.3.2","info"],"pid":1,"state":"yellow","message":"Status changed from uninitialized to yellow - Waiting for Elasticsearch","prevState":"uninitialized","prevMsg":"uninitialized"}\n{"type":"log","@timestamp":"2018-08-20T16:40:16Z","tags":["status","plugin:searchprofiler@6.3.2","info"],"pid":1,"state":"yellow","message":"Status changed from uninitialized to yellow - Waiting for Elasticsearch","prevStat